In [ ]:
# install packages
try:
    __import__('numpy')
    print('numpy is already installed')
except ImportError:
    print("package not found installing")
    %pip install numpy

try:
    __import__('pygame')
    print('numpy is already installed')
except ImportError:
    print("package not found installing")
    %pip install pygame

   ---------------------------------------- 0.0/10.6 MB ? eta -:--:--
   ---------------------------------------- 0.0/10.6 MB 640.0 kB/s eta 0:00:17
    --------------------------------------- 0.1/10.6 MB 2.1 MB/s eta 0:00:05
   - -------------------------------------- 0.5/10.6 MB 4.5 MB/s eta 0:00:03
   --- ------------------------------------ 1.0/10.6 MB 6.1 MB/s eta 0:00:02
   ----- ---------------------------------- 1.5/10.6 MB 7.8 MB/s eta 0:00:02
   -------- ------------------------------- 2.2/10.6 MB 8.6 MB/s eta 0:00:01
   ----------- ---------------------------- 3.1/10.6 MB 10.2 MB/s eta 0:00:01
   --------------- ------------------------ 4.0/10.6 MB 12.1 MB/s eta 0:00:01
   ------------------- -------------------- 5.2/10.6 MB 13.8 MB/s eta 0:00:01
   ----------------------- ---------------- 6.3/10.6 MB 15.4 MB/s eta 0:00:01
   ------------------------- -------------- 6.8/10.6 MB 15.0 MB/s eta 0:00:01
   ------------------------------ --------- 8.2/10.6 MB 16.4 MB/s eta 0:00:0

In [3]:
import numpy as np
import random
import sys
import math
import pygame

In [5]:
# Game constants
ROWS = 6
COLS = 7
PLAYER = 'X'
AI = 'O'
EMPTY = ' '

# Colors (RGB)
BLUE = (0, 0, 255)
BLACK = (0, 0, 0)
RED = (255, 0, 0)
YELLOW = (255, 255, 0)

SQUARESIZE = 100
RADIUS = int(SQUARESIZE / 2 - 5)

# Initialize board
def create_board():
    return np.full((ROWS, COLS), EMPTY, dtype=str)

# Check if column is valid for placement
def is_valid_location(board, col):
    return board[0][col] == EMPTY

# Get the next open row in the column
def get_next_open_row(board, col):
    for r in range(ROWS-1, -1, -1):
        if board[r][col] == EMPTY:
            return r

# Drop a piece into the board
def drop_piece(board, row, col, piece):
    board[row][col] = piece

# Check if the game has been won
def winning_move(board, piece):
    # Check horizontal
    for c in range(COLS-3):
        for r in range(ROWS):
            if all(board[r][c+i] == piece for i in range(4)):
                return True

    # Check vertical
    for c in range(COLS):
        for r in range(ROWS-3):
            if all(board[r+i][c] == piece for i in range(4)):
                return True

    # Check positive diagonals
    for c in range(COLS-3):
        for r in range(ROWS-3):
            if all(board[r+i][c+i] == piece for i in range(4)):
                return True

    # Check negative diagonals
    for c in range(COLS-3):
        for r in range(3, ROWS):
            if all(board[r-i][c+i] == piece for i in range(4)):
                return True
    return False

# Evaluate a window of 4 pieces
def evaluate_window(window, piece):
    score = 0
    opp_piece = PLAYER if piece == AI else AI

    if window.count(piece) == 4:
        score += 100
    elif window.count(piece) == 3 and window.count(EMPTY) == 1:
        score += 5
    elif window.count(piece) == 2 and window.count(EMPTY) == 2:
        score += 2

    if window.count(opp_piece) == 3 and window.count(EMPTY) == 1:
        score -= 4

    return score

# Heuristic evaluation function
def score_position(board, piece):
    score = 0

    # Score center column
    center_array = [board[i][COLS//2] for i in range(ROWS)]
    center_count = center_array.count(piece)
    score += center_count * 3

    # Score horizontal
    for r in range(ROWS):
        row_array = list(board[r])
        for c in range(COLS-3):
            window = row_array[c:c+4]
            score += evaluate_window(window, piece)

    # Score vertical
    for c in range(COLS):
        col_array = [board[r][c] for r in range(ROWS)]
        for r in range(ROWS-3):
            window = col_array[r:r+4]
            score += evaluate_window(window, piece)

    # Score positive diagonal
    for r in range(ROWS-3):
        for c in range(COLS-3):
            window = [board[r+i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    # Score negative diagonal
    for r in range(ROWS-3):
        for c in range(COLS-3):
            window = [board[r+3-i][c+i] for i in range(4)]
            score += evaluate_window(window, piece)

    return score

# Minimax algorithm with Alpha-Beta pruning
def minimax(board, depth, alpha, beta, maximizingPlayer):
    valid_locations = [col for col in range(COLS) if is_valid_location(board, col)]
    random.shuffle(valid_locations)  # Add randomness for similar moves

    if depth == 0 or winning_move(board, PLAYER) or winning_move(board, AI):
        if winning_move(board, AI):
            return (None, 100000000000000)
        elif winning_move(board, PLAYER):
            return (None, -10000000000000)
        else:
            # Evaluate based on current player's perspective
            current_piece = AI if maximizingPlayer else PLAYER
            return (None, score_position(board, current_piece))

    if maximizingPlayer:
        value = -float('inf')
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, AI)
            new_score = minimax(b_copy, depth-1, alpha, beta, False)[1]
            if new_score > value:
                value = new_score
                column = col
            alpha = max(alpha, value)
            if alpha >= beta:
                break
        return column, value
    else:
        value = float('inf')
        column = random.choice(valid_locations)
        for col in valid_locations:
            row = get_next_open_row(board, col)
            b_copy = board.copy()
            drop_piece(b_copy, row, col, PLAYER)
            new_score = minimax(b_copy, depth-1, alpha, beta, True)[1]
            if new_score < value:
                value = new_score
                column = col
            beta = min(beta, value)
            if alpha >= beta:
                break
        return column, value

# AI's move selection
def get_ai_move(board):
    # First check if AI can win in next move
    for col in range(COLS):
        if is_valid_location(board, col):
            temp_board = board.copy()
            row = get_next_open_row(temp_board, col)
            drop_piece(temp_board, row, col, AI)
            if winning_move(temp_board, AI):
                return col

    # Block player's winning moves
    for col in range(COLS):
        if is_valid_location(board, col):
            temp_board = board.copy()
            row = get_next_open_row(temp_board, col)
            drop_piece(temp_board, row, col, PLAYER)
            if winning_move(temp_board, PLAYER):
                return col

    # Otherwise use minimax
    col, _ = minimax(board, 5, -float('inf'), float('inf'), True)
    return col

# Draw the board using Pygame
def draw_board(board, screen):
    for c in range(COLS):
        for r in range(ROWS):
            # Draw the board background
            pygame.draw.rect(screen, BLUE, (c*SQUARESIZE, r*SQUARESIZE+SQUARESIZE, SQUARESIZE, SQUARESIZE))
            # Draw the empty slot or piece
            piece = board[r][c]
            if piece == PLAYER:
                color = RED
            elif piece == AI:
                color = YELLOW
            else:
                color = BLACK
            pygame.draw.circle(screen, color, 
                               (int(c*SQUARESIZE + SQUARESIZE/2), int(r*SQUARESIZE + SQUARESIZE + SQUARESIZE/2)), 
                               RADIUS)
    pygame.display.update()

def main():
    pygame.init()

    width = COLS * SQUARESIZE
    height = (ROWS+1) * SQUARESIZE  # extra row at top for showing moves
    size = (width, height)
    screen = pygame.display.set_mode(size)
    pygame.display.set_caption("Connect 4")

    board = create_board()
    game_over = False
    turn = random.choice([PLAYER, AI])
    
    # Font for displaying messages
    myfont = pygame.font.SysFont("monospace", 50)

    draw_board(board, screen)
    pygame.display.update()

    while not game_over:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            # Player's turn: mouse motion draws piece on top
            if turn == PLAYER:
                if event.type == pygame.MOUSEMOTION:
                    pygame.draw.rect(screen, BLACK, (0, 0, width, SQUARESIZE))
                    posx = event.pos[0]
                    pygame.draw.circle(screen, RED, (posx, int(SQUARESIZE/2)), RADIUS)
                    pygame.display.update()

                if event.type == pygame.MOUSEBUTTONDOWN:
                    pygame.draw.rect(screen, BLACK, (0, 0, width, SQUARESIZE))
                    posx = event.pos[0]
                    col = int(math.floor(posx / SQUARESIZE))
                    
                    if is_valid_location(board, col):
                        row = get_next_open_row(board, col)
                        drop_piece(board, row, col, PLAYER)

                        if winning_move(board, PLAYER):
                            label = myfont.render("Player wins!", 1, RED)
                            screen.blit(label, (40,10))
                            game_over = True

                        turn = AI
                        draw_board(board, screen)

        # AI's turn
        if turn == AI and not game_over:
            # Small delay for effect
            pygame.time.wait(500)
            col = get_ai_move(board)
            if is_valid_location(board, col):
                row = get_next_open_row(board, col)
                drop_piece(board, row, col, AI)

                if winning_move(board, AI):
                    label = myfont.render("AI wins!", 1, YELLOW)
                    screen.blit(label, (40,10))
                    game_over = True

                turn = PLAYER
                draw_board(board, screen)

        # If game over, wait a few seconds before closing
        if game_over:
            pygame.time.wait(3000)

if __name__ == "__main__":
    main()
